<p align="right"><img src="https://upload.wikimedia.org/wikipedia/commons/8/85/TxDOT_logo_as_of_2023.svg" width="550"></p>

# TxDOT “Road-Bot” Mission
## 🚗 STUDENT TEMPLATE – Car Model Recognition

Welcome to the fill-in-the-blanks version of the **Car Model Recognition** notebook.

**Goal:** Teach an AI to tell a **Suzuki Swift** from a **Suzuki WagonR**.  Think of Road-Bot as a student. We’ll show it hundreds of photos labeled Swift or WagonR, and it’ll learn to tell which is which—like flash cards for cars.

**Why?** TxDOT technicians use these counts to plan parking-lot expansions and anti-congestion measures.  

You are now *Road-Bot Junior Engineers*—let’s get to work!  


<p align="right"><img src="https://www.cartoq.com/wp-content/uploads/2023/10/Maruti-Suzuki-WagonR-vs-Maruti-Suzuki-SwiftA.jpg" width="800"></p>


### 📝 How to Use This Notebook

🔧 You’ll be working through a series of simple, fill-in-the-blank tasks.

Here’s how to stay on track:

- Look for cells labeled **🚀 STUDENT TASK** — those are your action zones!
- After each one, run the **✅ QUICK CHECK** cell right below it to make sure everything’s working.
- If you see the message  
  `"Feel free to tweak earlier sliders & re-run."`  
  it means you're close — just try adjusting some values and run again.

✅ When everything checks out, the final cell will celebrate with a big green **ALL DONE!** banner.

> 💡 **Stuck?** Don’t worry — just unfold the hint right after the task for help.

---

**You're in control — explore, test, and have fun. You've got this! 🚗💨**







---
## 🚦 Mission Brief – Become a “Road-Bot” Engineer

TxDOT (Texas Department of Transportation) is building an AI helper called **Road-Bot**.  
Its first job: scan dash-cam footage and **count how many Suzuki Swifts vs. Suzuki WagonRs** are parked near schools.  
Those counts feed into *real* planning decisions—like how wide to make new pickup/drop-off lanes.

To turn raw pictures into trustworthy numbers, we’ll follow the same pipeline professional data-scientists use:

| Step | What happens | Why it matters |
|------|--------------|----------------|
| **1. Setup** (you just click ▶) | Install PyTorch & download the mini car-photo dataset. | Gives us tools + data in one place so everyone starts on equal footing. |
| **2. Tune the Basics** (🚀 TASK #1) | Choose batch size, learning rate, and number of epochs. | These dials control **how fast** and **how safely** the network learns. |
| **3. Augment the Data** (🚀 TASK #2) | Decide if we flip, rotate, or recolor training images. | Extra “looks” at each car teach the model to ignore camera angle & lighting. |
| **4. Build DataLoaders** | Package the photos into mini-batches PyTorch can read. | Feeds the network efficiently—like handing flash-cards in neat stacks. |
| **5. Pick a CNN Size** (🚀 TASK #3) | Tiny / Small / Medium model presets. | Balances accuracy vs. training time on a classroom laptop. |
| **6. Set the optimiser** (🚀 TASK #4) | Select Adam / RMSprop / SGD. | Controls *how* weights are updated; can speed up or steady training. |
| **7. Train & Watch** | The loop prints loss & accuracy each epoch. | You’ll *see* Road-Bot get better—often within 30 seconds. |
| **8. Quick Check #4** | If validation accuracy > 80 %, a big green **ALL DONE!** banner appears. | Confirms the model is good enough for TxDOT’s planning tool. |


**Your role:** After each *🚀 STUDENT TASK* cell, hit ▶ on the *✅ QUICK CHECK* right below it.  
When you hit the green banner, you’ve officially graduated to **Road-Bot Junior Engineer**. 🌟

---


In [ ]:
#@title ⚙️ Setup – RUN ONCE (≈2 min) ▸ installs libraries • downloads data { display-mode:"form" }
"""
What this cell does
───────────────────
✓ Installs PyTorch, torchvision, tqdm
✓ Downloads the ZIP (~90 MB) if not cached
✓ Unzips into /content/cars-wagonr-swift
✓ Defines global constants used by later cells
"""

# import urllib.request, zipfile, time, sys, os

# DATASET_URL = "https://www.dropbox.com/scl/fi/45gxs6ddi5h51to3bvpjx/cars-wagonr-swift.zip"
# ZIP_NAME    = "cars-wagonr-swift.zip"
# EXTRACT_DIR = "/content/data"

# # ── tiny spinner so Colab feels alive ─────────────────────────────────────
# def _spinner(msg="Setting up…"):
#     for ch in "|/-\\":
#         sys.stdout.write(f"\r{msg} {ch}")
#         sys.stdout.flush()
#         time.sleep(0.08)

try:
    # 1️⃣ install packages
    # _spinner("🔧 Installing PyTorch & friends")
    !pip -q install torch torchvision tqdm

    # 2️⃣ import libraries
    # _spinner("📦 Importing libraries")
    import torch, numpy as np, PIL, matplotlib.pyplot as plt
    import torchvision.transforms as transforms
    from tqdm.auto import tqdm

    # # 3️⃣ download + unzip dataset (once)
    # if not os.path.exists(EXTRACT_DIR):
    #     # _spinner("⬇️  Downloading dataset (~90 MB)")
    #     urllib.request.urlretrieve(DATASET_URL, ZIP_NAME)
    #     # _spinner("📂 Unzipping dataset")
    #     zipfile.ZipFile(ZIP_NAME).extractall(EXTRACT_DIR)

    print("\n✅ Environment ready!  Move to the first 🚀 STUDENT TASK.")
except Exception as e:
    print("\n❌ Setup failed:", e)
    raise  # re-throw so users notice

# ---------- fixed paths & constants ----------
TRAIN_DATA_PATH      = "data_full/train"
VALIDATION_DATA_PATH = "data_full/validation"
TEST_DATA_PATH       = "data_full/test"

WAGONEER_LABEL = "wagonr"
SWIFT_LABEL    = "swift"
IMAGE_SIZE     = (50, 50)

# 📘 **How Road-Bot Learns**

Think of Road-Bot as a student. Road-Bot learns by studying labeled photos—some show Suzuki Swifts, others show WagonRs.  We’ll show it hundreds of photos labeled Swift or WagonR, and it’ll learn to tell which is which—like flash cards for cars.

Imagine showing Road-Bot 16 flash cards at a time. Each time it sees a batch of photos, it tries to guess the car type, gets corrected, and adjusts its “brain.” After each group, it adjusts how it ‘thinks’ to improve accuracy. We repeat that process over and over—each full run through the whole dataset is one epoch.


> 📝 **What do these numbers mean?**  
> In this step, you’ll choose:
>
> • 📦 **Batch Size** – How many photos Road-Bot sees at once (like flipping flash cards).  (*Hint! 8–64 is laptop-friendly*).
>  
> • 🧠 **Learning Rate** – How big each adjustment is after an error. (*Hint! 0.001 is safe*)
>
> • 🔁 **Epochs** – How many times it goes through all the flash cards. *Hint! Choose between 5-10 to show visible progress without stalling the workshop.*


**Pro tip:** Start with the default values below, then tweak them and see what changes!



In [ ]:
#@title 🚀 STUDENT TASK #1 – Tune the basics { display-mode:"form" }
def _hyper():
  BATCH_SIZE = 8       #@param {type:"slider",  label:"📦 Batch size",   min:8, max:64, step:8}
  LEARNING_RATE = 0.001 #@param {type:"number",  label:"🚀 Learning rate", step:0.0005}
  EPOCHS = 3            #@param {type:"slider",  label:"🔁 Epochs",       min:3, max:20}
  return BATCH_SIZE, LEARNING_RATE, EPOCHS

BATCH_SIZE, LEARNING_RATE, EPOCHS = _hyper()


In [ ]:
#@title ✅ Quick Check #1 { display-mode:"form" }

assert 8 <= BATCH_SIZE <= 64,       "Try something between 8 and 64."
assert 1e-5 <= LEARNING_RATE <= 1,  "Learning rate looks off."
assert 3 <= EPOCHS <= 30,           "Pick 3-30 epochs."
print("🎉 Looks good! Move to the next cell.")


# 📸 How Road-Bot Sees the World

Before we teach Road-Bot, let’s **peek at the photos** it’ll be learning from.

We’ve already downloaded a mini dataset of **Suzuki Swift** and **Suzuki WagonR** pictures. Each image is labeled so Road-Bot knows what it’s looking at while it learns.

This step is like **flipping through a study guide before class starts** — we want to see what kind of examples Road-Bot will use to learn the difference between the two car types.

---

🧭 **What to look for:**

- Do the **Swift and WagonR** cars have noticeable shape or size differences?  
- Are the **photo angles** consistent or all over the place?  
- Do some images have tricky conditions like shadows or parked cars?

---

📘 **Why this matters:**  
The more you understand the data, the better you can **train, troubleshoot, and improve** Road-Bot later.  
This is your first glimpse of what Road-Bot will “see” as it learns to drive its neural engine.

> *Pro tip:* You can come back and re-run this preview cell anytime you want to inspect the data!


In [ ]:
#@title 👀 Meet the data – run to view a sample { display-mode:"form" }

from pathlib import Path
import random, matplotlib.pyplot as plt, PIL

def _show_sample(root, cls, n=6):
  files = list(Path(root/cls).glob("*"))
  plt.figure(figsize=(9,6))
  for i,f in enumerate(random.sample(files, n)):
      img = PIL.Image.open(f)
      plt.subplot(2,3,i+1); plt.imshow(img); plt.axis("off")
  plt.tight_layout()

_show_sample(Path("/content/data/train"), "swift")


# 📈 How Road-Bot Gets Better at Seeing

Some photos are bright, some are dark. Some cars are angled left, others right.  
To make Road-Bot more flexible, we can **augment the training images**—flipping, rotating, or adjusting the color just a bit.

This is like showing flash cards from different angles or lighting so the model won’t get confused in the real world.

---

🎛️ **Choose an augmentation pack** to help Road-Bot:
- Flip cars left or right (helps with angle variation)
- Rotate or zoom slightly (simulates imperfect photos)
- Change brightness or hue (trains Road-Bot for cloudy or sunny days)

> 🧪 Try different packs and see which one improves performance!


In [ ]:
#@title 🚀 STUDENT TASK #2 – Choose an augmentation pack { display-mode:"form" }
aug_pack = "None"  #@param ["None", "Flip Only", "Flip + Rotate", "Flip + ColorJitter"]

import torchvision.transforms as T
if aug_pack == "None":
    aug_list = []
elif aug_pack == "Flip Only":
    aug_list = [T.RandomHorizontalFlip()]
elif aug_pack == "Flip + Rotate":
    aug_list = [T.RandomHorizontalFlip(),
                T.RandomRotation(20)]
else:  # Flip + ColorJitter
    aug_list = [T.RandomHorizontalFlip(),
                T.RandomRotation(20),
                T.ColorJitter(brightness=.2, contrast=.2, saturation=.2)]

# 🔒 🏗️ BUILD DATASET & DATALOADERS  (no edits needed) { display-mode:"form" }
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from pathlib import Path

# --- 1. Compose transforms ------------------
train_tfms = T.Compose(aug_list + [                # ← set in STUDENT TASK #2
    T.Resize(IMAGE_SIZE),                          # (150,150)
    T.ToTensor()
])
test_tfms  = T.Compose([
    T.Resize(IMAGE_SIZE),
    T.ToTensor()
])

# --- 2. Point to the folders -----------------
train_ds = ImageFolder(Path(TRAIN_DATA_PATH),      transform=train_tfms)
val_ds   = ImageFolder(Path(VALIDATION_DATA_PATH), transform=test_tfms)
test_ds  = ImageFolder(Path(TEST_DATA_PATH),       transform=test_tfms)

# --- 3. Wrap in DataLoaders ------------------
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)

# print(f"🎉 DataLoaders ready – "
#       f"{len(train_loader)} train batches | "
#       f"{len(val_loader)} val batches | "
#       f"{len(test_loader)} test batches")


In [ ]:
#@title ✅ Quick Check #2 { display-mode:"form" }

import torchvision.transforms as T, PIL, random, matplotlib.pyplot as plt
tmp = T.Compose(aug_list + [T.ToTensor()])

sample_path = "/content/data/train/swift"
img = PIL.Image.open(random.choice(list(Path(sample_path).glob("*"))))
plt.imshow(tmp(img).permute(1,2,0)); plt.title("Augmentation preview"); plt.axis("off")
print("🎉 Looks like the transform works!  Move on ➜")


# 🏋🏼‍♀️ How Powerful Should Road-Bot’s Brain Be?

A Convolutional Neural Network (CNN) is like Road-Bot’s brain.  
You can choose a **Tiny**, **Small**, or **Medium** version.

Think of it like car engines:

- **Tiny (3 layers)** – Fast and simple. Good for short races.
- **Small (4 layers)** – Balanced. Fast and smart.
- **Medium (5 layers)** – Powerful, but takes longer to train.

> 🏎️ Bigger isn’t always better—pick based on how much time you have and how complex the task is.


In [ ]:
#@title 🚀 STUDENT TASK #3 – Select a CNN size { display-mode:"form" }
model_size = "Tiny (3 layers)"  #@param ["Tiny (3 layers)", "Small (4 layers)", "Medium (5 layers)"]

import torch.nn as nn, torch

def make_net(choice):
    if choice == "Tiny (3 layers)":
        layers = [3,16,32]      # output chans per conv layer
    elif choice == "Small (4 layers)":
        layers = [3,16,32,64]
    else:                        # Medium
        layers = [3,16,32,64,128]

    convs, chans = [], layers[0]
    for out_ch in layers[1:]:
        convs += [nn.Conv2d(chans, out_ch, 3, padding=1),
                  nn.BatchNorm2d(out_ch),
                  nn.ReLU(),
                  nn.MaxPool2d(2)]
        chans = out_ch
    return nn.Sequential(*convs,
                         nn.Flatten(),
                         nn.Linear(chans * (IMAGE_SIZE[0]//(2**len(layers[1:])))**2, 2))

net = make_net(model_size).to("cpu")    # GPU later if available
print(net)


In [ ]:
#@title ✅ Quick Check #3 { display-mode:"form" }

param_cnt = sum(p.numel() for p in net.parameters())
assert 10_000 < param_cnt < 5_000_000, "Param count off – did you change something by accident?"
print(f"🎯 Network has {param_cnt:,} trainable parameters – good to go!")


# 📖 How Road-Bot Learns from Mistakes

Each time Road-Bot guesses wrong, it adjusts its weights to do better next time.  
The **optimiser** controls *how* those adjustments happen—like the gears in a race car’s transmission.

Choose from:

- ⚙️ **SGD** – The original, classic learning strategy.
- ⚡ **Adam** – Fast and smooth. Great all-round choice.
- 🧭 **RMSprop** – Like Adam, but sometimes better for noisy data.

> 🚦 Try one, then test again to see which gives the best accuracy!


In [ ]:
#@title 🚀 STUDENT TASK #4 – Set the optimiser { display-mode:"form" }
optimiser = "SGD"  #@param ["Adam", "RMSprop", "SGD"]

import torch.optim as optim, torch
criterion = nn.CrossEntropyLoss()
opt = {"Adam":optim.Adam,
       "RMSprop":optim.RMSprop,
       "SGD":optim.SGD}[optimiser](net.parameters(), lr=LEARNING_RATE)

# @markdown <details><summary>💡 <b>Which optimiser?</b> (Click to expand)</summary>
# @markdown
# @markdown Optimisers help Road-Bot adjust its weights after each mistake—like picking how to correct your answer after getting a flashcard wrong.
# @markdown
# @markdown • **Adam** – Fast, balanced, and usually the best all-around choice for beginners.
# @markdown
# @markdown &nbsp;&nbsp;&nbsp;&nbsp; ✨ Think of it as a smart cruise control that adapts to each curve in the road.
# @markdown
# @markdown • **SGD (Stochastic Gradient Descent)** – The original method. Slower, but useful in academic research and fine-tuned projects.
# @markdown
# @markdown &nbsp;&nbsp;&nbsp;&nbsp; 🔧 Like shifting gears manually—more work, but gives control.
# @markdown
# @markdown • **RMSprop** – Similar to Adam, but better for problems with noisy or bumpy data.
# @markdown
# @markdown &nbsp;&nbsp;&nbsp;&nbsp; 🧭 Helps Road-Bot stay steady on uneven terrain.
# @markdown
# @markdown
# @markdown If you're curious, try the others and compare results!
# @markdown </details>



# 🔒 Hidden — students just run
from tqdm.auto import tqdm
def run_epoch(loader, train=True):
    net.train() if train else net.eval()
    loss_all = correct = 0
    desc = "BATCH-TRAIN" if train else "BATCH-VALIDATION"
    with torch.set_grad_enabled(train):
        for x,y in tqdm(loader, desc=desc):
            if train: opt.zero_grad()
            out = net(x); loss = criterion(out,y)
            if train: loss.backward(); opt.step()
            loss_all += loss.item()*x.size(0)
            correct  += (out.argmax(1)==y).sum().item()
    return loss_all/len(loader.dataset), correct/len(loader.dataset)

hist = {"tr_loss":[], "tr_acc":[], "val_loss":[], "val_acc":[]}
for ep in tqdm(range(EPOCHS), desc="EPOCH"):
    tl,ta = run_epoch(train_loader, True)
    vl,va = run_epoch(val_loader,   False)
    hist["tr_loss"].append(tl); hist["tr_acc"].append(ta)
    hist["val_loss"].append(vl); hist["val_acc"].append(va)
    print(f"[{ep+1}/{EPOCHS}] loss:{tl:.3f} acc:{ta:.3f}  |  val_loss:{vl:.3f} val_acc:{va:.3f}")
best_val_acc = max(hist["val_acc"])


In [ ]:
#@title ✅ Quick Check #4 – Celebrate results! { display-mode:"form" }

from IPython.display import Markdown, HTML, display

if best_val_acc > 0.80:
    # Text summary (1 line)
    display(Markdown(f"<h2 style='color:green'>🎉 ALL DONE! "
                     f"You reached {best_val_acc:.1%} accuracy.</h2>"))

    # HTML banner
    banner_html = """
    <p style="background:#d4edda;
              color:#155724;
              padding:14px 0;
              border-radius:6px;
              font-weight:bold;
              font-size:1.3em;
              text-align:center;">
      🎉 ALL&nbsp;DONE!
    </p>"""
    display(HTML(banner_html))
else:
    print(f"🔄 Final validation accuracy {best_val_acc:.1%}. "
          "Feel free to tweak earlier sliders & re-run.")


# ✨ Reflection Time!
1. What surprised you most about teaching Road-Bot?
2. If you could teach Road-Bot to recognize anything, what would it be?
3. What new word or concept did you learn today?


Type your answers in here:


*   List item
*   List item

